In [1]:
import sys
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
sys.path.append('/projects/trans_scratch/validations/workspace/szong/Cervical/notebook')
import sz_toolbox as szt

# plotly.offline.init_notebook_mode(connected=True)
# Standard plotly imports
import plotly.plotly as py
import pandas as pd
# import plotly
import plotly.graph_objs as go
from plotly.offline import iplot
# cufflinks binds plotly directly to python dataframe
# import cufflinks as cf
# cufflinks.go_offline()
# # Set the global theme
# cf.set_config_file(world_readable=True, theme='pearl', offline=True)
# plotly.offline.init_notebook_mode(connected=True)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


production runs documented here:

https://www.bcgsc.ca/jira/browse/PIPE-1144

In [2]:
wkdir = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/validation/'
f = 'strelka_classic_vcf_files.txt'

In [ ]:
def parse_strelka_vcf(vcf, impacts):
    impacts = '|'.join(impacts)
    df = pd.read_csv(vcf, comment='#', sep='\t', header=None, low_memory=False)
    patient = vcf.split('/')[4]
    df = df[[0,1,3,4,7]]
    df.columns = ['chr', 'pos', 'ref', 'alt', 'effect']
#     df = df[(df['effect'].str.contains("HIGH"))|(df['effect'].str.contains("MODERATE"))|(df['effect'].str.contains("LOW"))]
    df = df[df['effect'].str.contains(impacts, regex=True)]
#     print(df)
    if not df.empty:
        df['impact'], df['impact_type'], df['gene'] = df['effect'].apply(lambda x: parse_effect(x)).str.split('@').str
        df['patient'] = patient
    df = df.drop('effect', axis=1)
    return df

In [3]:
# keep it easy for now pick HIGH, MODERATE and then LOW
def parse_effect(line):
    effs = line.split('EFF=')[1].split(',')
    #     extract impact, impact_type and gene
    effs = ['@'.join(list(np.array(re.split('\(|\|',ef))[[0,1,6]])) for ef in effs if ('HIGH' in  ef) or ('MODERATE' in ef) or ('LOW' in ef) or ('MODIFIER' in ef)]
#     effs = list(set(effs))#????
#     print(effs)
    high = [ef for ef in effs if 'HIGH' in ef]
    moderate = [ef for ef in effs if 'MODERATE' in ef]
    low = [ef for ef in effs if 'LOW' in ef]
    modifier = [ef for ef in effs if 'MODIFIER' in ef]
    if high:
        anno = high[0]
    elif moderate:
        anno = moderate[0]
    elif low:
        anno = low[0]
    elif modifier:
        anno = modifier[0]
    else:
        print('ERROR!')
#     make sure the genes have the same name
    genes = [ef.split('@')[2] for ef in effs]
    
    return anno

In [5]:
vcfs = pd.read_csv(f'{wkdir}{f}', header=None)[0].tolist()

In [6]:
snvvcfs = [i for i in vcfs if 'snvs' in i]
len(snvvcfs)
snvvcfs[:2]
indelvcfs = [i for i in vcfs if 'indel' in i]
len(indelvcfs)

87

['/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02025/hg19a/GSH/A93956_A93945/strelka/61000/bwa/results/passed.somatic.snvs.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf',
 '/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02044/hg19a/GSH/A93957_A93946/strelka/61001/bwa/results/passed.somatic.snvs.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf']

87

In [7]:
df = pd.DataFrame({'snv':snvvcfs, 'indel': indelvcfs})
df['patient'] = df.snv.apply(lambda x: x.split('/')[4])
df['pat2'] = df.indel.apply(lambda x: x.split('/')[4])

assert (df.patient == df.pat2).all() == True
df = df.drop('pat2', axis=1)
df.columns = ['strelka_snv_vcf', 'strelka_indel_vcf', 'patient']
df.head(2)

,strelka_snv_vcf,strelka_indel_vcf,patient
0,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,HTMCP-03-06-02025
1,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,HTMCP-03-06-02044


In [8]:
dfmg = pd.DataFrame()
for ix, row in df.iterrows():
    patient = ix
    snv_vcf = row['strelka_snv_vcf']
    indel_vcf = row['strelka_indel_vcf']
#     snv_df = szt.parse_strelka_vcf(snv_vcf)
    snv_df = parse_strelka_vcf(snv_vcf, ['HIGH', 'MODERATE', 'LOW', 'MODIFIER'])
#     indel_df = szt.parse_strelka_vcf(indel_vcf)
    indel_df = parse_strelka_vcf(indel_vcf, ['HIGH', 'MODERATE', 'LOW', 'MODIFIER'])
    mdf = pd.concat([snv_df, indel_df], sort=False)
    mdf = mdf.drop_duplicates(keep='first')
    if dfmg.empty:
        dfmg = mdf
    else:
        dfmg = pd.concat([dfmg, mdf], sort=False)

In [9]:
dfmg[:2]
dfmg.shape

,chr,pos,ref,alt,impact,impact_type,gene,patient
0,1,914517,C,T,SYNONYMOUS_CODING,LOW,C1orf170,HTMCP-03-06-02025
1,1,1919953,C,T,NON_SYNONYMOUS_CODING+SPLICE_SITE_REGION,MODERATE,C1orf222,HTMCP-03-06-02025


(92654, 8)

In [10]:
recur_genes = dfmg[['gene', 'patient']].groupby('gene')['patient'].nunique().sort_values(ascending=False)
recur_genes[:2]

gene
              87
AF146191.4    87
Name: patient, dtype: int64

In [11]:
dfmg.impact_type.unique()

array(['LOW', 'MODERATE', 'MODIFIER', 'HIGH'], dtype=object)

In [12]:
def uniquejoin(x):
    return ';'.join(x.unique())

In [13]:
dfs = dfmg.astype(str).groupby(['gene', 'chr', 'pos', 'ref', 'alt']).agg({'patient':[uniquejoin, 'nunique'], 'impact_type':uniquejoin, 'impact':uniquejoin})
dfs = dfs.sort_values(('patient','nunique'), ascending=False).reset_index()
dfs.columns = ['gene', 'chr', 'pos', 'ref', 'alt', 'patient', 'num_patients_variant', 'impact_type', 'impact']
dfs.head(2)

,gene,chr,pos,ref,alt,patient,num_patients_variant,impact_type,impact
0,RP11-391M20.1,9,68723937,C,T,HTMCP-03-06-02061;HTMCP-03-06-02121;HTMCP-03-0...,22,MODIFIER,UPSTREAM
1,,GL000198.1,89425,C,G,HTMCP-03-06-02062;HTMCP-03-06-02067;HTMCP-03-0...,15,MODIFIER,INTERGENIC


In [14]:
gene_patients_counts = dfmg.astype(str).groupby(['gene']).agg({'patient':'nunique'}).sort_values('patient', ascending=False)

In [15]:
gene_patients_counts.head(2)

,patient
gene,
,87
RP11-262H14.1,87


In [16]:
dff = pd.merge(dfs, gene_patients_counts, left_on='gene', right_index=True, how='left')
dff = dff.sort_values(['patient_y', 'gene'], ascending=False)

In [17]:
dff[dff.gene=='PIK3CA'].head(2)

,gene,chr,pos,ref,alt,patient_x,num_patients_variant,impact_type,impact,patient_y
7,PIK3CA,3,178936091,G,A,HTMCP-03-06-02062;HTMCP-03-06-02082;HTMCP-03-0...,14,MODERATE,NON_SYNONYMOUS_CODING,29
934,PIK3CA,3,178936082,G,A,HTMCP-03-06-02061;HTMCP-03-06-02121;HTMCP-03-0...,5,MODERATE,NON_SYNONYMOUS_CODING,29


In [18]:
dff.shape

(69804, 10)

In [19]:
dff.to_csv(f'{wkdir}/strelka_HMLM_summary.txt.new', sep='\t', index=False)